In [10]:
# mypy: allow-untyped-defs
import os
import random
import json
from pathlib import Path
from collections import Counter


import numpy as np
from PIL import Image


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms, datasets


from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score


# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


# Make cudnn deterministic (may slow training slightly)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cpu


In [11]:
config = {
'data_dir': 'characters_train', # path to folder that contains class subfolders
'img_size': 128,
'batch_size': 64,
'num_workers': 4,
'epochs': 10,
'lr': 1e-3,
'weight_decay': 1e-4,
'val_size': 0.15, # validation portion of the *training* set (stratified)
'save_path': 'model.pth',
'label_map_path': 'label_map.json'
}


# safe checks
assert os.path.exists(config['data_dir']), f"data_dir {config['data_dir']} not found"

In [12]:
train_transform = transforms.Compose([
transforms.Resize((config['img_size'], config['img_size'])),
transforms.RandomHorizontalFlip(),
transforms.RandomRotation(12),
transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


val_transform = transforms.Compose([
transforms.Resize((config['img_size'], config['img_size'])),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# ImageFolder will read folders as classes
full_dataset = datasets.ImageFolder(config['data_dir'], transform=train_transform)


# Save label map (index->class name) for inference
label_map = {v: k for k, v in full_dataset.class_to_idx.items()} # reverse mapping
with open(config['label_map_path'], 'w') as f:
    json.dump(label_map, f)


print('Classes found:', full_dataset.classes)
print('Total images:', len(full_dataset))

Classes found: ['abraham_grampa_simpson', 'agnes_skinner', 'apu_nahasapeemapetilon', 'barney_gumble', 'bart_simpson', 'carl_carlson', 'charles_montgomery_burns', 'chief_wiggum', 'cletus_spuckler', 'comic_book_guy', 'disco_stu', 'edna_krabappel', 'fat_tony', 'gil', 'groundskeeper_willie', 'homer_simpson', 'kent_brockman', 'krusty_the_clown', 'lenny_leonard', 'lionel_hutz', 'lisa_simpson', 'maggie_simpson', 'marge_simpson', 'martin_prince', 'mayor_quimby', 'milhouse_van_houten', 'miss_hoover', 'moe_szyslak', 'ned_flanders', 'nelson_muntz', 'otto_mann', 'patty_bouvier', 'principal_skinner', 'professor_john_frink', 'rainier_wolfcastle', 'ralph_wiggum', 'selma_bouvier', 'sideshow_bob', 'sideshow_mel', 'snake_jailbird', 'troy_mcclure', 'waylon_smithers']
Total images: 16764


In [13]:
targets = np.array(full_dataset.targets)
sss = StratifiedShuffleSplit(n_splits=1, test_size=config['val_size'], random_state=SEED)
train_idx, val_idx = next(sss.split(np.zeros(len(targets)), targets))


# Subsets that use different transforms (train has augmentation, val has val_transform)
train_dataset = Subset(datasets.ImageFolder(config['data_dir'], transform=train_transform), train_idx)
val_dataset = Subset(datasets.ImageFolder(config['data_dir'], transform=val_transform), val_idx)


print('Train images:', len(train_dataset), 'Val images:', len(val_dataset))

Train images: 14249 Val images: 2515


In [14]:
train_targets = [full_dataset.targets[i] for i in train_idx]
class_counts = Counter(train_targets)
print('Train class distribution (label_id:count):', class_counts)


# compute weights: inverse-freq
class_weights = {cls: 1.0/count for cls, count in class_counts.items()}
sample_weights = [class_weights[t] for t in train_targets]


sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)


train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], sampler=sampler,
num_workers=config['num_workers'], pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False,
num_workers=config['num_workers'], pin_memory=True)

Train class distribution (label_id:count): Counter({15: 1527, 28: 989, 27: 988, 20: 921, 4: 913, 22: 878, 17: 820, 32: 813, 6: 812, 25: 734, 7: 671, 0: 621, 37: 597, 2: 424, 16: 339, 9: 320, 11: 311, 29: 244, 18: 211, 24: 167, 41: 123, 21: 88, 14: 82, 3: 72, 36: 71, 5: 67, 35: 61, 31: 49, 23: 49, 33: 44, 39: 37, 8: 32, 34: 31, 1: 29, 38: 27, 30: 22, 13: 19, 12: 19, 26: 12, 10: 6, 40: 6, 19: 3})


In [15]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


num_classes = len(full_dataset.classes)
model = SimpleCNN(num_classes=num_classes).to(device)
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, af

In [16]:
counts = [class_counts[i] for i in range(num_classes)]
class_weights_tensor = torch.tensor([sum(counts)/c for c in counts], dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [ ]:
from tqdm import tqdm


best_f1 = 0.0
best_epoch = -1
history = {'train_loss': [], 'val_loss': [], 'val_f1': []}


for epoch in range(1, config['epochs'] + 1):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f'Epoch {epoch} - train'):
        imgs = imgs.to(device)
        labels = labels.to(device)


        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        
        running_loss += loss.item() * imgs.size(0)


    train_loss = running_loss / len(train_loader.sampler)


# validation
    model.eval()
    val_loss = 0.0
    preds = []
    trues = []
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f'Epoch {epoch} - val'):
            imgs = imgs.to(device)
            labels = labels.to(device)
            logits = model(imgs)
            loss = criterion(logits, labels)
            val_loss += loss.item() * imgs.size(0)


    batch_preds = torch.argmax(logits, dim=1).cpu().numpy()
    preds.extend(batch_preds.tolist())
    trues.extend(labels.cpu().numpy().tolist())
    
    
    val_loss = val_loss / len(val_loader.dataset)
    val_f1 = f1_score(trues, preds, average='macro')
    
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)
    
    
    print(f"Epoch {epoch}: train_loss={train_loss:.4f} val_loss={val_loss:.4f} val_f1={val_f1:.4f}")
    
    
    # scheduler step (we want to maximize f1)
    scheduler.step(val_f1)
    
    
    # save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch
        torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': full_dataset.class_to_idx,
        'img_size': config['img_size']
        }, config['save_path'])
        print(f'Best model saved at epoch {epoch} with val_f1={val_f1:.4f}')


print(f'Finished. Best val_f1={best_f1:.4f} at epoch {best_epoch}')

Epoch 1 - train:   0%|          | 0/223 [00:00<?, ?it/s]C:\Users\KiuStudnet\PycharmProjects\SpringField\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1 - train:   2%|▏         | 4/223 [00:27<16:28,  4.51s/it]  

In [ ]:
print('Model saved to', config['save_path'])
print('Label map saved to', config['label_map_path'])
print('To run inference, open inference.ipynb which expects the saved model and label_map.json')